In [ ]:
import polars as pl
import psycopg2
import pandas as pd
import import_ipynb
from Utils import *
import os
import time
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()

connection_params = {
    'dbname': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port': os.getenv("DB_PORT")
}

print("Parámetros de conexión cargados.")

In [ ]:
# Función para normalizar texto
def normalize_text(series):
    return series.str.to_lowercase().str.strip_chars().str.replace_all(r'[.,;:!?()\[\]{}¿¡-]', '')

# Conectar a la base de datos
conn = psycopg2.connect(**connection_params)
cur = conn.cursor()

start_time = time.time()
print("Inicio de la transferencia con Polars.")
# Truncar tablas de prueba para estado limpio
#cur.execute("TRUNCATE TABLE paciente_clean_test, infolab_clean_test;")
#conn.commit()
print("Tablas de prueba truncadas.")

In [ ]:
# Cargar datos a DataFrames usando Pandas y convirtiendo a Polars
try:
    import pandas as pd
    df_paciente_pd = pd.read_sql("SELECT * FROM paciente", conn)
    df_infolab_pd = pd.read_sql("SELECT * FROM infolab", conn)
    df_paciente_clean_pd = pd.read_sql("SELECT * FROM paciente_clean_test_2", conn)
    df_infolab_clean_pd = pd.read_sql("SELECT * FROM infolab_clean_test_2", conn)
    # Convertir a Polars
    df_paciente = pl.from_pandas(df_paciente_pd)
    df_infolab = pl.from_pandas(df_infolab_pd)
    df_paciente_clean = pl.from_pandas(df_paciente_clean_pd)
    df_infolab_clean = pl.from_pandas(df_infolab_clean_pd)
    print("Datos cargados exitosamente.")
except Exception as e:
    print(f"Error al cargar datos: {e}")
    raise

In [ ]:
# Transferir pacientes únicos a paciente_clean_test (con actualización si existe)
try:
    df_paciente_unique = df_paciente.unique(subset=['id_paciente'])
    if not df_paciente_unique.is_empty():
        cols = ['nombre1', 'nombre2', 'nombre3', 'apellido1', 'apellido2', 'tipo_de_identificacion', 'identificacion', 'sexo', 'fecha_nacimiento', 'edad', 'municipio_residencia', 'codigo_residencia', 'barrio_residencia', 'direccion', 'id_paciente', 'etiqueta', 'fecha_ingreso_sistema']
        data = df_paciente_unique.select(cols).to_pandas().values.tolist()
        
        cur.executemany("""
            INSERT INTO paciente_clean_test_2 (nombre1, nombre2, nombre3, apellido1, apellido2, tipo_de_identificacion, identificacion, sexo, fecha_nacimiento, edad, municipio_residencia, codigo_residencia, barrio_residencia, direccion, id_paciente, etiqueta, fecha_ingreso_sistema)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (id_paciente) DO UPDATE SET
                nombre1 = EXCLUDED.nombre1,
                nombre2 = EXCLUDED.nombre2,
                nombre3 = EXCLUDED.nombre3,
                apellido1 = EXCLUDED.apellido1,
                apellido2 = EXCLUDED.apellido2,
                tipo_de_identificacion = EXCLUDED.tipo_de_identificacion,
                identificacion = EXCLUDED.identificacion,
                sexo = EXCLUDED.sexo,
                fecha_nacimiento = EXCLUDED.fecha_nacimiento,
                edad = EXCLUDED.edad,
                municipio_residencia = EXCLUDED.municipio_residencia,
                codigo_residencia = EXCLUDED.codigo_residencia,
                barrio_residencia = EXCLUDED.barrio_residencia,
                direccion = EXCLUDED.direccion,
                etiqueta = EXCLUDED.etiqueta,
                fecha_ingreso_sistema = EXCLUDED.fecha_ingreso_sistema;
        """, data)
        
        conn.commit()
                
        df_paciente_clean_pd = pd.read_sql("SELECT * FROM paciente_clean_test_2", conn)
        df_paciente_clean = pl.from_pandas(df_paciente_clean_pd)
        
        print(f"✓ Pacientes insertados/actualizados: {len(df_paciente_unique)}")
        print(f"  Total en paciente_clean_test: {len(df_paciente_clean)}")        
    else:
        print("No hay pacientes nuevos para insertar en paciente_clean_test.")
except Exception as e:
    print(f"Error al transferir pacientes: {e}")
    conn.rollback()
    raise

In [ ]:
# Normalizar columnas clave para infolab
cols_infolab = ['servicio', 'muestra_remitida', 'descripcion_macroscopica', 'descripcion_microscopica', 'diagnostico']
df_infolab_norm = df_infolab.with_columns([
    normalize_text(pl.col(c).fill_null('')).alias(f"{c}_norm") for c in cols_infolab
])

df_infolab_clean_norm = df_infolab_clean.with_columns([
    normalize_text(pl.col(c).fill_null('')).alias(f"{c}_norm") for c in cols_infolab
])

print("Normalización completada.")

In [ ]:
# Filtrar infolab nuevos: relacionados con pacientes existentes y no duplicados
# Primero, deduplicar df_infolab por columnas normalizadas
df_infolab_dedup = df_infolab_norm.unique(
    subset=['id_paciente'] + [f"{c}_norm" for c in cols_infolab]
)

# Filtrar solo los relacionados con pacientes existentes
df_infolab_relacionados = df_infolab_dedup.join(
    df_paciente_clean.select('id_paciente'),
    on='id_paciente',
    how='inner'
)

print(f"Registros de infolab relacionados con pacientes existentes: {len(df_infolab_relacionados)}")

# Crear clave compuesta para comparación
df_infolab_relacionados = df_infolab_relacionados.with_columns([
    pl.concat_str([
        pl.col('id_paciente').cast(pl.Utf8),
        pl.lit('|'),
        pl.col('servicio_norm').cast(pl.Utf8),
        pl.lit('|'),
        pl.col('muestra_remitida_norm').cast(pl.Utf8),
        pl.lit('|'),
        pl.col('descripcion_macroscopica_norm').cast(pl.Utf8),
        pl.lit('|'),
        pl.col('descripcion_microscopica_norm').cast(pl.Utf8),
        pl.lit('|'),
        pl.col('diagnostico_norm').cast(pl.Utf8)
    ]).alias('_key')
])

# Crear keys existentes
if not df_infolab_clean_norm.is_empty():
    df_infolab_clean_norm = df_infolab_clean_norm.with_columns([
        pl.concat_str([
            pl.col('id_paciente').cast(pl.Utf8),
            pl.lit('|'),
            pl.col('servicio_norm').cast(pl.Utf8),
            pl.lit('|'),
            pl.col('muestra_remitida_norm').cast(pl.Utf8),
            pl.lit('|'),
            pl.col('descripcion_macroscopica_norm').cast(pl.Utf8),
            pl.lit('|'),
            pl.col('descripcion_microscopica_norm').cast(pl.Utf8),
            pl.lit('|'),
            pl.col('diagnostico_norm').cast(pl.Utf8)
        ]).alias('_key')
    ])
    keys_existentes = set(df_infolab_clean_norm.select('_key').to_series().to_list())
    print(f"Keys existentes en infolab_clean: {len(keys_existentes)}")
else:
    keys_existentes = set()
    print("infolab_clean está vacío, todos los registros relacionados serán insertados.")

# Filtrar registros nuevos
df_infolab_filtered = df_infolab_relacionados.filter(
    ~pl.col('_key').is_in(keys_existentes)
).drop(['_key'] + [f"{c}_norm" for c in cols_infolab])

print(f"Registros filtrados para infolab_clean_test: {len(df_infolab_filtered)}")

In [ ]:
try:
    if not df_infolab_filtered.is_empty():
        cols_to_insert = [
            'fecha_toma_muestra', 'fecha_ingreso', 'fecha_informe', 'entidad', 'eps',
            'servicio', 'muestra_remitida', 'descripcion_macroscopica',
            'descripcion_microscopica', 'diagnostico', 'comentario', 'id_paciente',
            'archivo', 'fuente', 'cie10', 'edad', 'sexo', 'diagnostico_clinico',
            'observacion', 'medico_ordenante', 'medico_remitente'
        ]
        
        # Convertir a lista de tuplas para executemany
        data = df_infolab_filtered.select(cols_to_insert).to_pandas().values.tolist()
        
        cur.executemany("""
            INSERT INTO infolab_clean_test_2 (fecha_toma_muestra, fecha_ingreso, fecha_informe, entidad, eps, servicio, muestra_remitida, descripcion_macroscopica, descripcion_microscopica, diagnostico, comentario, id_paciente, archivo, fuente, cie10, edad, sexo, diagnostico_clinico, observacion, medico_ordenante, medico_remitente)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, data)
        
        conn.commit()
        print(f"✓ Registros insertados en infolab_clean_test: {len(df_infolab_filtered)}")
    else:
        print("No hay registros nuevos para insertar en infolab_clean_test.")
        
except Exception as e:
    conn.rollback()
    print(f"✗ Error al insertar infolab: {e}")
    raise

In [ ]:
try:
    conn.commit()
    end_time = time.time()
    print(f"\n{'='*60}")
    print(f"Transferencia completada exitosamente en {end_time - start_time:.2f} segundos.")
    print(f"Tiempo total de ejecución con Polars: {end_time - start_time:.2f} segundos.")
    print(f"{'='*60}")
except Exception as e:
    print(f"Error al confirmar cambios: {e}")
    conn.rollback()
    raise
finally:
    cur.close()
    conn.close()
    print("Conexión cerrada.")